In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

batch_size = 128
epochs = 10
learning_rate = 1e-3
# Change this to your data directory
data_dir = "/Users/little1d/Desktop/Code/Generative-Models/data"

In [32]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [33]:
# data

train_dataset = datasets.MNIST(
    root=data_dir, train=True, transform=transforms.ToTensor(), download=True
)
test_dataset = datasets.MNIST(
    root=data_dir, train=False, transform=transforms.ToTensor()
)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
train_dataset[0][0].shape

torch.Size([1, 28, 28])

In [ ]:
import time

model = MLP(input_size=784, hidden_size=128, output_size=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(epochs):
    start_time = time.time()
    total_loss = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 28 * 28)
        # forward
        outputs = model(images)
        loss = criterion(outputs, labels)
        total_loss += loss.item()
        # backward
        optimizer.zero_grad()
        loss.backward()
        # update
        optimizer.step()

    end_time = time.time()
    epoch_time = end_time - start_time
    avg_loss = total_loss / len(train_loader)
    print(
        f"Epoch {epoch + 1}: Time taken {epoch_time:.2f} seconds, Average loss: {avg_loss:.4f}"
    )

Epoch 1: 用时 2.26 秒, 平均损失: 0.3488
Epoch 2: 用时 2.27 秒, 平均损失: 0.1584
Epoch 3: 用时 2.28 秒, 平均损失: 0.1107
Epoch 4: 用时 2.20 秒, 平均损失: 0.0853
Epoch 5: 用时 2.26 秒, 平均损失: 0.0667
Epoch 6: 用时 2.21 秒, 平均损失: 0.0546
Epoch 7: 用时 2.32 秒, 平均损失: 0.0445
Epoch 8: 用时 2.54 秒, 平均损失: 0.0366
Epoch 9: 用时 2.25 秒, 平均损失: 0.0305
Epoch 10: 用时 2.47 秒, 平均损失: 0.0247


In [35]:
# eval
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.view(-1, 28 * 28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Accuracy of the network on the 10000 test images: {100 * correct / total}%")

Accuracy of the network on the 10000 test images: 97.75%
